# Obtain transcripts from seekalpha.com

In [14]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import os


In [15]:
sectors = ['basic-materials',
          'conglomerates',
          'consumer-goods',
          'financial',
          'healthcare',
          'industrial-goods',
          'services',
          'technology',
          'utilities']

for sec in sectors:
    if not os.path.exists('Transcripts/{}'.format(sec)):
        os.mkdir('Transcripts/{}'.format(sec))

In [16]:
sec =  'technology'
os.path.exists('Transcript/{}'.format(sec))

False

## Log in to seekalpha

In [17]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches",["enable-automation"])
driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options,)

driver.get("https://seekingalpha.com/account/login")
email = driver.find_element_by_id("login_user_email")
email.send_keys("zhouxing@uchicago.edu")
password = driver.find_element_by_id("login_user_password")
password.send_keys("Xz19980114!")
#button = driver.find_element_by_xpath("//input[@value='Sign in']")
driver.find_element_by_xpath("//div[@class='register_button']/input[@value='Sign in']").click()

[WDM] - Looking for [chromedriver 84.0.4147.30 mac64] driver in cache 
[WDM] - File found in cache by path [/Users/zhou/.wdm/drivers/chromedriver/84.0.4147.30/mac64/chromedriver]


In [18]:
# driver.get("https://seekingalpha.com/article/1594412-siemens-aktiengesellschaft-management-discusses-q3-2013-results-earnings-call-transcript")
# content = driver.find_element_by_id("a-body")
# heading = driver.find_element_by_tag_name('h1').text
# html = str(content.get_attribute('innerHTML'))
# reg = r'<p class="p p[1-9][0-9]{0,1}">(.+?)<\/p>'
# url_lst = filter(lambda x: x[:8]!="<strong>",re.findall(reg,html))
# text = list(url_lst)
# with open("./text/{}.txt".format(heading),"a+") as f:
#     for line in text:
#         f.write("{}\n".format(line))
# f.close()

In [19]:
# def get_stock_name_heading_time(url):
#     driver.get(url)
#     name = driver.find_element_by_xpath("//div[@class='snap-data']/ul[@class='table']/li").text
#     stock_name = str(name).split('\n')[1]
#     heading = driver.find_element_by_tag_name('h1').text
#     time_slot = re.findall(r"Q[0-9] 2[0-9]{3}",heading)
#     return stock_name,heading,time_slot

In [20]:
def find_time_name_from_title(text):
    time_lst = re.findall(r"Q[0-9] 2[0-9]{3}",text)
    name_lst = re.findall(r'\((.+?)\)',text)
    return time_lst,name_lst

In [21]:
def write_urls(num_page_range,sector):
    cnt = 0
    filepath = "Transcripts/{}/urls.txt".format(sector)
    if os.path.exists(filepath):
        os.remove(filepath)
    with open(filepath,"a+") as f:
        for page in num_page_range:
            driver.get("https://seekingalpha.com/earnings/earnings-call-transcripts/{}?sector={}".format(page,sector))
            content = driver.find_elements_by_xpath("//a[@sasource='earnings-center-transcripts_article']")
            for i in range(len(content)):
                title = content[i].text
                time_lst,name_lst = find_time_name_from_title(title)
                if len(time_lst)!=1 or len(name_lst)!=1:
                    continue
                Time = time_lst[0].replace(" ","")
                Name = name_lst[0]
                url = content[i].get_attribute('href')
                f.write("{};{};{}\n".format(Name,Time,url))
                cnt+=1
    f.close()
    print("urls:",cnt)
                        

In [22]:
def write_transcipt(name,time,url,sector):
    
    dirpath = "Transcripts/{}/Transcripts".format(sector)
    if not os.path.exists(dirpath):
        os.mkdir(dirpath)
        
    filepath = "Transcripts/{}/Transcripts/{}_{}.txt".format(sector,name,time)
    
    if os.path.exists(filepath):
        return 
    
    driver.get(url)
    content = driver.find_element_by_id("a-body")
    html = str(content.get_attribute('innerHTML'))
    reg = r'<p class="p p[1-9][0-9]{0,1}">(.+?)<\/p>'
    url_lst = filter(lambda x: x[:8]!="<strong>",re.findall(reg,html))
    text = list(url_lst)
    
    with open(filepath,"a+") as f:
        for line in text:
            f.write("{}\n".format(line))
    f.close()
    


In [29]:
sec_pages = {"healthcare":42,
            "financial":52,
            "consumer-goods":24,
             "conglomerates":2,
             "services":48,
             "technology":66,
             "utilities":8,
             'industrial-goods':26
            }
sector = input("sector:")
write_urls(range(sec_pages[sector]),sector)

urls: 56


In [30]:
filepath = "Transcripts/{}/urls.txt".format(sector)
with open(filepath,'rb') as f:
    res = f.read().decode('ascii').split('\n')
    for idx,i in enumerate(res):
        name,time,url = i.split(';')
        try:
            write_transcipt(name,time,url,sector)
            if idx%20==0:
                print("finished:",idx)
        except:
            print("error:",idx)
            

finished: 0
finished: 20
finished: 40


ValueError: not enough values to unpack (expected 3, got 1)